# Estimate Binomial draw probabilility

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-1.4020427180880297

Use a flat priors (the default, omitted) for α

In [5]:
P = TransformedLogDensity(as((α = as𝕀,)), p)
∇P = ADgradient(:ForwardDiff, P);

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0012 s/step ...done
MCMC, adapting ϵ (25 steps)
6.6e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0025 s/step ...done
MCMC, adapting ϵ (100 steps)
7.7e-6 s/step ...done
MCMC, adapting ϵ (200 steps)
6.8e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
8.0e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
1.1e-5 s/step ...done
MCMC (1000 steps)
6.0e-6 s/step ...done


(DynamicHMC.NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([-0.265391], -3.48794, 1, AdjacentTurn, 0.941814, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.477765], -3.36251, 1, DoubledTurn, 0.919748, 1), NUTS_Transition{Array{Float64,1},Float64}([-0.217862], -3.25004, 1, DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([0.353133], -2.92006, 2, DoubledTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.0913404], -5.47528, 2, DoubledTurn, 0.633665, 3), NUTS_Transition{Array{Float64,1},Float64}([0.532646], -3.66472, 2, DoubledTurn, 0.875113, 3), NUTS_Transition{Array{Float64,1},Float64}([0.768737], -3.20201, 1, DoubledTurn, 0.931539, 1), NUTS_Transition{Array{Float64,1},Float64}([0.768737], -6.24945, 1, AdjacentTurn, 0.504881, 3), NUTS_Transition{Array{Float64,1},Float64}([1.49791], -4.89926, 1, AdjacentTurn, 0.773922, 3), NUTS_Transition{Array{Float64,1},Float64}([1.1181], -4.74134, 1, DoubledTurn, 1.0, 1)  …  NUTS_Transiti

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

400.5824066836524

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.93, min/25%/median/75%/max: 0.15 0.89 0.97 1.0 1.0
  termination: AdjacentTurn => 35% DoubledTurn => 65%
  depth: 1 => 67% 2 => 33%


check the mean

In [11]:
mean(posterior_α)

0.5649052829139227

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*